In [1]:
#import Packages required for the Analysis
import pandas as pd
import numpy as np
import re 
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import nltk 
import random 
from nltk.corpus import RegexpTokenizer as regextoken
from nltk import word_tokenize
import string
import operator
import collections
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB , BernoulliNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2
from collections import Counter
from collections import defaultdict
from sklearn.feature_selection import  SelectKBest, chi2
from nltk.corpus import wordnet
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
import pickle


In [2]:
# Reading Data 
import pandas as pd
data=pd.read_csv("supervised_data.csv")
print(data.head(n=5))
print("The shape is:", data.shape)


  INCIDENT_NUMBER                                            Summary  \
0    INC006647448  Build Task - Request Desktop - Request Flexdes...   
1    INC006647449  Build Task - Request Desktop - Request Flexdes...   
2    INC006647450  Build Task - Request Desktop - Request Flexdes...   
3    INC006647451  Build Task - Request Desktop - Request Flexdes...   
4    INC006647452                       Office crashing during work    

  BUSINESS_RULE_GROUP_NAME Account_name  
0                  DESKTOP        INTEL  
1                  DESKTOP        INTEL  
2                  DESKTOP        INTEL  
3                  DESKTOP        INTEL  
4                MS OFFICE        INTEL  
The shape is: (3769292, 4)


In [4]:
data=data.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
data.shape

(3753184, 4)

In [5]:
data.columns

Index(['INCIDENT_NUMBER', 'Summary', 'BUSINESS_RULE_GROUP_NAME',
       'Account_name'],
      dtype='object')

In [6]:
#Required to convert in to string
data['Description']=data['Summary'].astype(str)
# Data CLeaning
data['Description']=data['Description'].astype(str)
data['Description'] = data['Description'].str.replace('[','')
data['Description'] = data['Description'].str.replace(']','')
data['Description'] = data['Description'].str.replace(',','')
data['Description'] = data['Description'].str.replace('\'','')
data.head()

,INCIDENT_NUMBER,Summary,BUSINESS_RULE_GROUP_NAME,Account_name,Description
0,INC006647448,Build Task - Request Desktop - Request Flexdes...,DESKTOP,INTEL,Build Task - Request Desktop - Request Flexdes...
1,INC006647449,Build Task - Request Desktop - Request Flexdes...,DESKTOP,INTEL,Build Task - Request Desktop - Request Flexdes...
2,INC006647450,Build Task - Request Desktop - Request Flexdes...,DESKTOP,INTEL,Build Task - Request Desktop - Request Flexdes...
3,INC006647451,Build Task - Request Desktop - Request Flexdes...,DESKTOP,INTEL,Build Task - Request Desktop - Request Flexdes...
4,INC006647452,Office crashing during work,MS OFFICE,INTEL,Office crashing during work


In [7]:
# English Library
english_vocab = set(w.lower() for w in nltk.corpus.words.words())


In [8]:
# Text pre processing with Lemmatization - We need to pass POS tag  then only results will be good
#We are passing only Noun , Verb , adjective and adverbs
allowed_word_types = ["N","V","J","R"]
wordnet_lemmatizer = WordNetLemmatizer()

In [9]:
# Function that returns POS tagging in required formate
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None # for easy if-statement 

In [10]:
# Function that lemmitize the word 
def lemm(document):
    all_words=[]
    stopword = stopwords.words('english')
    stop_words = set(stopwords.words('english'))
    domainSpecificStopwords = ["issue","quick","check","inform","unable","folder",
                             "number","question","limit","max","team","wrong","http",
                             "time","run","meet","review","move","due","custom","urgent",
                             "interface","standard","move","external","setup","conversion","information",
                             "request","error"]

    stop_words.update(domainSpecificStopwords) 
    tokenizer = regextoken(r'\w+')
    words = tokenizer.tokenize(document)
    pos = nltk.pos_tag(words)
    for w in pos:
            if w[1][0] in allowed_word_types and len(w[0])>1 and w[0] not in stop_words:
                wntag = get_wordnet_pos(w[1])
                w_c=wordnet_lemmatizer.lemmatize(w[0].lower(),pos=wntag)
                if w_c not in english_vocab:
                    all_words.append(w[0].lower())
                else:
                    all_words.append(w_c.lower())
                
    return all_words

In [27]:
# Applying Lemmitization on data Column 'Summary' 
import time
start_time = time.time()
#data['Processed_text_Lemm'] = ""
data['Processed_text_Lemm'] = data.apply(lambda row: lemm(row['Description']),axis=1)

print("This Block took", (round((time.time() - start_time)/60.0, 2)), " minutes to run")

KeyboardInterrupt: 

In [10]:
# Data CLeaning
data['Processed_text_Lemm_POS']=data['Processed_text_Lemm'].astype(str)
data['Processed_text_Lemm_POS'] = data['Processed_text_Lemm_POS'].str.replace('[','')
data['Processed_text_Lemm_POS'] = data['Processed_text_Lemm_POS'].str.replace(']','')
data['Processed_text_Lemm_POS'] = data['Processed_text_Lemm_POS'].str.replace(',','')
data['Processed_text_Lemm_POS'] = data['Processed_text_Lemm_POS'].str.replace('\'','')
data.head(n=10)

,ID,Summary,BUSINESS_RULE_GROUP_NAME,Description,Processed_text_Lemm,Processed_text_Lemm_POS
0,INC1473133,Laptop Old Wants New Upgrade,LAPTOP,Laptop Old Wants New Upgrade,"[laptop, old, want, new, upgrade]",laptop old want new upgrade
1,INC1466907,"Client hopi04 bk.pri.bms.com, Class Linux Unix...",UNIX,Client hopi04 bk.pri.bms.com Class Linux Unix ...,"[client, hopi04, bk, pri, bms, com, class, lin...",client hopi04 bk pri bms com class linux unix ...
2,INC1554410,Unable to login password management,PASSWORD MANAGEMENT,Unable to login password management,"[unable, login, password, management]",unable login password management
3,INC1562791,Laptop Letter G not working anymore,LAPTOP,Laptop Letter G not working anymore,"[laptop, letter, work, anymore]",laptop letter work anymore
4,INC1374899,user is unable to use her phone port on laptop,PHONE,user is unable to use her phone port on laptop,"[user, use, phone, port, laptop]",user use phone port laptop
5,INC1450227,workstation is not compatible with my desktop,DESKTOP,workstation is not compatible with my desktop,"[workstation, compatible, desktop]",workstation compatible desktop
6,INC1331746,unable to access vpn from home,VPN,unable to access vpn from home,"[access, vpn, home]",access vpn home
7,INC1237309,Service NOT running Symantec CCS Agent Produc...,CCS,Service NOT running Symantec CCS Agent Produc...,"[service, not, run, symantec, ccs, agent, prod...",service not run symantec ccs agent production
8,INC1451790,**CRITICAL** Database Instance WM CON.WORLD on...,DATABASE,**CRITICAL** Database Instance WM CON.WORLD on...,"[critical, database, instance, wm, con, world,...",critical database instance wm con world usabhb...
9,INC1460610,Password reset for GTS 14A,GTS,Password reset for GTS 14A,"[password, reset, gts]",password reset gts


In [ ]:
data_categories=data[data.BUSINESS_RULE_GROUP_NAME!="NOT ASSIGNED"]
data_not_assigned=data[data.BUSINESS_RULE_GROUP_NAME=="NOT ASSIGNED"]

In [11]:
# Model Training

In [12]:
#data.to_csv("BMS_processed_test.csv")

In [13]:
data_chi_POS=data

In [14]:
# Stratified Sampling
X = data_chi_POS.Processed_text_Lemm_POS
y = data_chi_POS.BUSINESS_RULE_GROUP_NAME
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(X, y)
for train_index, test_index in sss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train_POS, X_test_POS = X[train_index], X[test_index]
    y_train_POS, y_test_POS = y[train_index], y[test_index]

TRAIN: [111936 288818  82138 ... 253585 133197 104269] TEST: [ 97611 125915 276999 ... 130695 297888  25661]


In [15]:
# Stratified Sampling for Train set and development
X_tra = pd.DataFrame(X_train_POS.reset_index())['Processed_text_Lemm_POS']
y_tra = pd.DataFrame(y_train_POS.reset_index())['BUSINESS_RULE_GROUP_NAME']
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
sss.get_n_splits(X_tra, y_tra)
for train_index, test_index in sss.split(X_tra, y_tra):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train_POS_dev, X_test_POS_val = X_tra[train_index], X_tra[test_index]
    y_train_POS_dev, y_test_POS_val = y_tra[train_index], y_tra[test_index]

TRAIN: [245851 181725 125953 ...  26690 275764 165502] TEST: [199145 251454 120882 ... 215907  82639 145924]


In [16]:
# Create dummies for classes
y_train_d=pd.get_dummies(y_train_POS_dev)
print(y_train_d.shape,y_train_POS_dev.shape)

(230445, 415) (230445,)


In [17]:
# Converting TF
count_vect_POS_D = CountVectorizer(binary=True)
X_train_counts_POS_D = count_vect_POS_D.fit_transform(X_train_POS_dev)
print(X_train_counts_POS_D.shape)

(230445, 28593)


In [18]:
#loop for extracting each feature having features > 8.54/0.001 chi sqaure/prob
import time
start_time = time.time()
No_Top_features=100
cols = list(y_train_d.columns)
print(len(cols))
Features_final=[]
ChiSqauare_final=[]
for col in cols:
    chi2score = chi2(X_train_counts_POS_D,y_train_d[col])[0]
    Chi2prob = chi2(X_train_counts_POS_D,y_train_d[col])[1]
    Features = count_vect_POS_D.get_feature_names()
    CHS_tbl=pd.DataFrame({'Features':Features, 'Chisqaure':chi2score , 'CHi_Sqaure_Prob':Chi2prob})
    CHS_tbl_Cutoff=CHS_tbl.loc[(CHS_tbl["Chisqaure"] > 10.83)]
    if len(CHS_tbl_Cutoff)>No_Top_features:
        Tbl_imp=CHS_tbl_Cutoff.sort_values(by='Chisqaure' , ascending=False).head(No_Top_features)
    else:
        Tbl_imp=CHS_tbl_Cutoff
    Features_final.append(Tbl_imp['Features'].tolist())
    ChiSqauare_final.append(Tbl_imp['Chisqaure'].tolist())
    
print("This Block took", (round((time.time() - start_time)/60.0, 2)), " minutes to run")

415
This Block took 1.06  minutes to run


In [19]:
fea_imp = [item for sublist in Features_final for item in sublist]
Chi_fea_imp = [item for sublist in ChiSqauare_final for item in sublist]
print(len(fea_imp))
print(len(Chi_fea_imp))
tbl_Feat_Chi_Imp=pd.DataFrame({'Features':fea_imp , 'Chisqaure':Chi_fea_imp})
print(tbl_Feat_Chi_Imp.shape)
feat_chi_Imp=pd.DataFrame(tbl_Feat_Chi_Imp.groupby(['Features'],as_index=False, sort=False)['Chisqaure'].max())
print(feat_chi_Imp.shape)
feat_chi_Imp.head()

33903
33903
(33903, 2)
(16868, 2)


,Features,Chisqaure
0,account,157352.409973
1,lock,17999.091510
2,unlock,25944.414698
3,directory,10184.703055
4,lan,51821.796198


In [22]:
#Linear SVC - Optimal K 
# To find out Best K features
start_time = time.time()
k_val=feat_chi_Imp.shape[0]
scores_BNB = []
Features_Tbl_final_BNB=0
for k in range(100, int(k_val/2) , 100):
    Features_Tbl_final_BNB=feat_chi_Imp.sort_values('Chisqaure', ascending=False).head(k)
    # Uni gram
    text_clf_CHi_POS = Pipeline([('vect', CountVectorizer(vocabulary=Features_Tbl_final_BNB.Features)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC(penalty="l1", dual=False,tol=1e-3)),])
    # Training the Model
    text_clf_CHi_POS.fit(X_train_POS_dev, y_train_POS_dev)
    # Scoring the Model
    predicted = text_clf_CHi_POS.predict(X_test_POS_val)
    scores_BNB.append(f1_score(y_test_POS_val, predicted, average='weighted'))

print("This Block took", (round((time.time() - start_time)/60.0, 2)), " minutes to run")

C:\Users\manoj.gundreddy\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\manoj.gundreddy\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\manoj.gundreddy\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\manoj.gundreddy\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\manoj.gundr

This Block took 508.42  minutes to run


In [23]:
Kvalues_BNB=list(range(100,int(k_val/2),100))
df_BNB = pd.DataFrame({'K_values':Kvalues_BNB , 'F1-Score':scores_BNB})
TopK_Val_BNB=df_BNB.sort_values(by='F1-Score' , ascending=False).head(n=10)
K_OPT_BNB=int(TopK_Val_BNB.iloc[0,1])
print(K_OPT_BNB)

8300


In [24]:
#Linear SVC
# Uni gram by using Optimal Features value
Features_Tbl_final_BNB=feat_chi_Imp.sort_values('Chisqaure', ascending=False).head(K_OPT_BNB)

text_clf_CHi_POS_D = Pipeline([('vect', CountVectorizer(vocabulary=Features_Tbl_final_BNB.Features)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',  LinearSVC(penalty="l1", dual=False,tol=1e-3)),])
# Training the Model
text_clf_CHi_POS_D.fit(X_train_POS, y_train_POS)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...max_iter=1000,
     multi_class='ovr', penalty='l1', random_state=None, tol=0.001,
     verbose=0))])

In [25]:
# Scoring the Model
predicted = text_clf_CHi_POS_D.predict(X_test_POS)
from sklearn import metrics
print(metrics.classification_report(y_test_POS, predicted))
print(f1_score(y_test_POS, predicted, average='weighted') )

                                                                precision    recall  f1-score   support

                                                       ACCOUNT       0.98      0.99      0.98       892
                                                       ADAPTOR       1.00      0.98      0.99        65
                                                         ADOBE       0.99      0.99      0.99       149
                                                      AIRWATCH       1.00      1.00      1.00        30
                                                        ALERTS       1.00      0.99      0.99      1720
                                                           ALM       0.97      0.97      0.97        34
                                                    ANYCONNECT       1.00      1.00      1.00        61
                                                        APACHE       1.00      1.00      1.00        51
                                                         ARIBA 

0.9651186018663624


In [35]:
#by using calibrated clsssifier 
text_clf_CHi_POS_D_new = Pipeline([('vect', CountVectorizer(vocabulary=Features_Tbl_final_BNB.Features)),
                     ('tfidf', TfidfTransformer()),
                     ('clf',  CalibratedClassifierCV(LinearSVC(penalty="l1", dual=False,tol=1e-3))),])

In [36]:
# Training the Model
text_clf_CHi_POS_D_new.fit(X_train_POS, y_train_POS)
# Scoring the Model
predicted = text_clf_CHi_POS_D_new.predict(X_test_POS)
from sklearn import metrics
print(metrics.classification_report(y_test_POS, predicted))
print(f1_score(y_test_POS, predicted, average='weighted') )

                                                                precision    recall  f1-score   support

                                                       ACCOUNT       0.98      0.98      0.98       892
                                                       ADAPTOR       1.00      0.97      0.98        65
                                                         ADOBE       0.99      0.99      0.99       149
                                                      AIRWATCH       1.00      1.00      1.00        30
                                                        ALERTS       1.00      0.99      0.99      1720
                                                           ALM       0.97      0.97      0.97        34
                                                    ANYCONNECT       1.00      1.00      1.00        61
                                                        APACHE       1.00      1.00      1.00        51
                                                         ARIBA 

0.9650853431584305


In [ ]:
# Pickle the model

In [23]:
# Dump the trained decision tree classifier with Pickle
model_classification_pkl_filename = 'model_classification.pkl'

# Open the file to save as pkl file
model_classification_pkl = open(model_classification_pkl_filename, 'wb')

#dump the file
pickle.dump(text_clf_CHi_POS_D, model_classification_pkl,protocol=2)

# Close the pickle instances
model_classification_pkl.close()

In [24]:
# To load back the pickle
# Loading the saved decision tree model pickle
model_classification_pkl = open(model_classification_pkl_filename, 'rb')
model_classification_model = pickle.load(model_classification_pkl)
print("Loaded SVM tree model :: ", model_classification_model)

Loaded Decision tree model ::  Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...alse, use_idf=True)), ('clf', BernoulliNB(alpha=1, binarize=0.0, class_prior=None, fit_prior=True))])
